In [1]:
import numpy as np      
import matplotlib.pyplot as plt 
import scipy.io.wavfile 
import subprocess
import librosa
import librosa.display
import IPython.display as ipd

from pathlib import Path, PurePath   
from tqdm.notebook import tqdm

## Utility functions

In [2]:
def convert_mp3_to_wav(audio:str) -> str:  
    """Convert an input MP3 audio track into a WAV file.

    Args:
        audio (str): An input audio track.

    Returns:
        [str]: WAV filename.
    """
    if audio[-3:] == "mp3":
        wav_audio = audio[:-3] + "wav"
        if not Path(wav_audio).exists():
                subprocess.check_output(f"ffmpeg -i {audio} {wav_audio}", shell=True)
        return wav_audio
    
    return audio

def plot_spectrogram_and_picks(track:np.ndarray, sr:int, peaks:np.ndarray, onset_env:np.ndarray) -> None:
    """[summary]

    Args:
        track (np.ndarray): A track.
        sr (int): Aampling rate.
        peaks (np.ndarray): Indices of peaks in the track.
        onset_env (np.ndarray): Vector containing the onset strength envelope.
    """
    times = librosa.frames_to_time(np.arange(len(onset_env)),
                            sr=sr, hop_length=HOP_SIZE)

    plt.figure()
    ax = plt.subplot(2, 1, 2)
    D = librosa.stft(track)
    librosa.display.specshow(librosa.amplitude_to_db(np.abs(D), ref=np.max),
                            y_axis='log', x_axis='time')
    plt.subplot(2, 1, 1, sharex=ax)
    plt.plot(times, onset_env, alpha=0.8, label='Onset strength')
    plt.vlines(times[peaks], 0,
            onset_env.max(), color='r', alpha=0.8,
            label='Selected peaks')
    plt.legend(frameon=True, framealpha=0.8)
    plt.axis('tight')
    plt.tight_layout()
    plt.show()

def load_audio_picks(audio, duration, hop_size):
    """[summary]

    Args:
        audio (string, int, pathlib.Path or file-like object): [description]
        duration (int): [description]
        hop_size (int): 

    Returns:
        tuple: Returns the audio time series (track) and sampling rate (sr), a vector containing the onset strength envelope
        (onset_env), and the indices of peaks in track (peaks).
    """
    try:
        track, sr = librosa.load(audio, duration=duration)
        onset_env = librosa.onset.onset_strength(track, sr=sr, hop_length=hop_size)
        peaks = librosa.util.peak_pick(onset_env, 10, 10, 10, 10, 0.5, 0.5)
    except Error as e:
        print('An error occurred processing ', str(audio))
        print(e)

    return track, sr, onset_env, peaks
    
    

## Settings

In [3]:
N_TRACKS = 1413
HOP_SIZE = 512
DURATION = 30 # TODO: to be tuned!
THRESHOLD = 5 # TODO: to be tuned!

In [4]:
data_folder = Path("./data/mp3s-32k/")
mp3_tracks = data_folder.glob("*/*/*.mp3")
tracks = data_folder.glob("*/*/*.wav")

In [5]:
songs = list(data_folder.glob("*/*/*.wav"))

In [6]:
songs

[PosixPath('data/mp3s-32k/radiohead/Amnesiac/10-Like_Spinning_Plates.wav'),
 PosixPath('data/mp3s-32k/radiohead/Amnesiac/04-You_and_Whose_Army.wav'),
 PosixPath('data/mp3s-32k/radiohead/Amnesiac/09-Hunting_Bears.wav'),
 PosixPath('data/mp3s-32k/radiohead/Amnesiac/08-Dollars_and_Cents.wav'),
 PosixPath('data/mp3s-32k/radiohead/Amnesiac/03-Pilk_Pull_Revolving_Doors.wav'),
 PosixPath('data/mp3s-32k/radiohead/Amnesiac/05-I_Might_Be_Wrong.wav'),
 PosixPath('data/mp3s-32k/radiohead/Amnesiac/11-Life_In_A_Glasshouse.wav'),
 PosixPath('data/mp3s-32k/radiohead/Amnesiac/01-Packt_Like_Sardines_In_A_Crush.wav'),
 PosixPath('data/mp3s-32k/radiohead/Amnesiac/02-Pyramid_Song.wav'),
 PosixPath('data/mp3s-32k/radiohead/Amnesiac/06-Knives_Out.wav'),
 PosixPath('data/mp3s-32k/radiohead/Amnesiac/07-Morning_Bell_Amnesiac.wav'),
 PosixPath('data/mp3s-32k/radiohead/Hail_to_the_Theif/11-A_Punchup_at_a_Wedding.wav'),
 PosixPath('data/mp3s-32k/radiohead/Hail_to_the_Theif/01-2_2_5_The_Lukewarm_.wav'),
 PosixPath(

## Preprocessing

## Audio signals

## Minhash

In [7]:
# TODO

In [8]:
from bitstring import BitArray
import pandas as pd
from collections import *
import pickle
import multiprocessing
from multiprocessing.dummy import Pool
import random

In [9]:
def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

In [10]:
def read_object(filename):
    with open(filename, 'rb') as file:
        data = pickle.load(file)
    return data

In [11]:
def timeOfPeaks(peaks, times):
    timesPeaks = []
    
    for i in peaks:
        timesPeaks.append(times[i])
    
    return timesPeaks

In [12]:
def fibonacci_hash_float(value:float, rand = False, hash_size = 15):

    value = BitArray(float=value, length=64)
    phi = (1 + 5 ** 0.5) / 2
    g = int(2 ** 64 /phi)


    value ^= value >> 61

    if(rand):
        value = int(g * value.float * np.random.random_sample(1))
    else:
        value = int(g * value.float)

    return int(str(value)[0:hash_size])

In [13]:
def make_fingerprints(audio, duration, hop = 0):
    track, sr, onset_env, peaks = load_audio_picks(audio, duration, HOP_SIZE)
    times = librosa.frames_to_time(np.arange(len(onset_env)), sr=sr, hop_length=HOP_SIZE)
    timesPeaks = timeOfPeaks(peaks, times)
    freqsP = [onset_env[i] for i in peaks]
    fingerprints = []
    
    if(hop != 0):
        sec = hop
        time=0
        count=0
        while(sec <= duration):

            idx = 0
            hashVal = 0
            count += 1

            while(time <=sec):

                if(timesPeaks[idx] < sec and timesPeaks[idx] > time):
                    hashVal ^= fibonacci_hash_float(freqsP[idx]) ^ hashVal


                time = timesPeaks[idx]

                if(idx+1 < len(freqsP)):
                    idx += 1
                else:
                    break

            fingerprints.append(hashVal)

            sec += hop

        fingerprints = list(filter(lambda a: a != 0, fingerprints))
    
    else:
        for fr in freqsP:
            fingerprints.append(fr)
   
    return fingerprints

In [14]:
def make_all_fingerprints(duration, hop = 0): 
    tempList = list()
    with Pool(multiprocessing.cpu_count()) as pool:
       
         with tqdm(total = 1413) as pbar:
            for i, el in enumerate(pool.imap(lambda song: make_fingerprints(song, duration, hop), songs)):
                tempList.append(el)
                pbar.update()
                
    return tempList

In [15]:
def getPeaks(audio, duration):
    _, _, _, peaks = load_audio_picks(audio, duration, HOP_SIZE);
    
    return (peaks, audio)

In [16]:
def get_all_peaks(songs, duration):
    
    peaks_of_songs = []
    
    with Pool(multiprocessing.cpu_count()) as pool:

        with tqdm(total = 1413) as pbar:
            for i, el in enumerate(pool.imap(lambda song: getPeaks(song, duration), songs)):
                peaks_of_songs.append(el)
                pbar.update()
                
    return peaks_of_songs

In [17]:
import random, copy, struct
import warnings
import numpy as np


class MinHash(object):

    def __init__(self, num_perm=128, seed=1, hashfunc=fibonacci_hash_float, vec = [], label = None):

        self._mersenne_prime = np.uint64((1 << 61) - 1)
        self._max_hash = np.uint64((1 << 32) - 1)
        self.seed = seed
        self.num_perm = num_perm
        self.hashfunc = hashfunc
        self.hashvalues = self._init_hashvalues(num_perm)
        self.permutations = self._init_permutations(num_perm)
        self.label = label
        
        if(len(vec)!=0):
            self.update_batch(vec, label)
            


    def _init_hashvalues(self, num_perm):
        return np.ones(num_perm, dtype=np.uint64)*self._max_hash

    def _init_permutations(self, num_perm):
        # Create parameters for a random bijective permutation function
        # that maps a 32-bit hash value to another 32-bit hash value.
        # http://en.wikipedia.org/wiki/Universal_hashing
        gen = np.random.RandomState(self.seed)
        return np.array([
            (gen.randint(1, self._mersenne_prime, dtype=np.uint64), gen.randint(0, self._mersenne_prime, dtype=np.uint64)) for _ in range(num_perm)
        ], dtype=np.uint64).T



    def update_batch(self, vec, label):
        self.label = label
        hv = np.array([self.hashfunc(_vec) for _vec in vec], dtype=np.uint64)
        a, b = self.permutations
        phv = np.bitwise_and(((hv * np.tile(a, (len(hv), 1)).T).T + b) % self._mersenne_prime, self._max_hash)
        self.hashvalues = np.vstack([phv, self.hashvalues]).min(axis=0)

    def jaccard(self, other):
        return float(np.count_nonzero(self.hashvalues==other.hashvalues)) / float(len(self.hashvalues))
    
    def __len__(self):
    
        return len(self.hashvalues)

In [18]:
class HashTable:
    def __init__(self):
        self.hash_table = defaultdict(list)
        
    def generate_hash(self, inp_vector):
        hashVal = 0
        for i in inp_vector:
            hashVal ^= fibonacci_hash_float(i) ^ hashVal
        return hashVal
            
    def setitem(self, vec, label):
        val = self.generate_hash(vec)
        self.hash_table[val].append(label)
        
        
    def getitem(self, inp_vec):
        hash_value = self.generate_hash(inp_vec)
        return self.hash_table.get(hash_value, [])
    
    def getTable(self):
        return(self.hash_table)

In [19]:
class LSH:
    def __init__(self, minhash_len, num_band):
        
        assert minhash_len % num_band == 0, "the choosen number of band does not hold the following assertion: minhash_len % num_band == 0"
        self.minhash_len = minhash_len
        self.num_band = num_band
        self.hash_tables = list()
        self.minhash_dict = dict()
        for i in range(self.num_band):
            self.hash_tables.append(HashTable())
            
            
    def addMinHash(self, minhash):
        self.minhash_dict[minhash.label] = minhash
        self.create_store_band(minhash.hashvalues, minhash.label)
            
    def create_store_band(self, vec, label):
        
        row_per_band = self.minhash_len // self.num_band
        
        subVec = []
        
        for i in range(0,self.minhash_len, row_per_band):
            subVec.append(vec[i:i+row_per_band])
        

        for band, table in zip(subVec, self.hash_tables):
            table.setitem(band, label)
            
    
    def create_band(self, vec):
        
        row_per_band = self.minhash_len // self.num_band
        
        subVec = []
        
        for i in range(0,self.minhash_len, row_per_band):
            subVec.append(vec[i:i+row_per_band])
        
        return subVec
    
            
    def query(self, minhash_query):
        subVec_query = self.create_band(minhash_query.hashvalues)
        match = set()
        similarities = []
        for table, band in zip(self.hash_tables, subVec_query):
            
            key = 0
            for i in band:
                key ^= fibonacci_hash_float(i) ^ key
                
            if(table.hash_table.get(key, "NA") != "NA"):
                match.update(tuple(table.hash_table.get(key)))
                
        for m in match:
            
            similarities.append((self.minhash_dict[m].jaccard(minhash_query), m))
            
            similarities = [ i for i in sorted(similarities, reverse=True)]
                
        return similarities
        

        


    def info(self):
        print("Numero di tabelle: " + str(self.num_tables))
        print("Elementi per tabella: " + str(len(self.hash_tables[0].hash_table)))

# Query test

take the fisrt query

In [35]:
#peaks = get_all_peaks(songs, duration=30)
#save_object(peaks, "./data/fingerprints/peaks_dur_30_hop_0.plk")
peaks = read_object("./data/fingerprints/peaks_dur_30_hop_0.plk")
minhashes = []

for fing, song in peaks:
    m = MinHash(num_perm=30, vec=fing, label=song)
    minhashes.append(m)
    


In [26]:
mq = MinHash(num_perm=30, vec=getPeaks("./data/queries/track1.wav", duration=10)[0], label="track1")

In [ ]:
out = []
for h in minhashes:
    out.append((h.jaccard(mq), h.label))


[ i for i in sorted(out, reverse=True) if i[0] > 0]

In [37]:
lsh = LSH(minhash_len=30, num_band=10)

for i in minhashes:
    lsh.addMinHash(i)


In [ ]:
lsh.query(mq)

In [ ]:
b = 10
r = 30 /b

t = (1/b)**(1/r)
t

In [ ]:
p1, o1 = getPeaks("./data/mp3s-32k/aerosmith/Aerosmith/03-Dream_On.wav", 50)
p2, o2 = getPeaks("./data/mp3s-32k/beatles/A_Hard_Day_s_Night/01-A_Hard_Day_s_Night.wav", 50)

In [ ]:
print("p1: " + str(len(p1)) + "     p2: " + str(len(p2)) + "\no1: " + str(len(o1)) + "   o2: " + str(len(o2)))